# Import and structure documents

In [1]:
from docx import Document

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
import re

In [76]:
filepaths = ['.\\data\\' + filename for filename in os.listdir('data')]

In [77]:
texts = ['\n'.join([par.text for par in Document(file).paragraphs]).strip() for file in filepaths]

In [78]:
corpus = pd.DataFrame(
    pd.Series(texts,
              name='texts')  #indexeket megtartja, de a felrobbantott iterable elemszámával megegyező observationt generál az explode()
    .str.split('\n').explode()).reset_index(names='doc_id')  #az indexek pedig így lesznek a doc_id-k

In [79]:
#Csak azokat tartjuk meg, amik megfelelnek a [valami]:[szöveg] logikának
corpus = corpus[corpus.texts.str.count(':') > 0]

In [80]:
corpus = pd.concat(
    [corpus['doc_id'],
     corpus['texts'].str.split(':', n=1)
     .apply(
         pd.Series)
     .rename({0: 'id', 1: 'text'}, axis=1)],
    axis=1)

In [81]:
kidobottak = corpus[corpus.id.str.count(' ') >= 2].copy()

In [82]:
corpus = corpus[corpus.id.str.count(' ') < 2]

In [83]:
#kézzel is szűrhető. itt már nem kell belemenni.
corpus = corpus[~corpus.id.isin(
    ['[Valaki felsóhajt', '[Kórusban ]', '[női hang]', '[férfi hang]', '[Férfihang]', '[többen összesúgnak',
     '[Valaki suttogva', '[Viccből]'])]

In [84]:
#csekkolni, hogy mi történt
pd.merge(pd.DataFrame(corpus.id.str.replace('[^a-zA-ZáéíóöőüűÁÉÍÓÖŐÜŰ\s]', '', regex=True).unique()),
         pd.DataFrame(corpus.id.unique()), how='outer', on=0, indicator=True)

,0,_merge
0,Dávid,left_only
1,A,both
2,D,both
3,Berta,both
4,D,both
...,...,...
111,Berta[?],right_only
112,Vera[?],right_only
113,Dóri?,right_only
114,Me1,right_only


In [85]:
corpus.id = corpus.id.str.replace('[^a-zA-ZáéíóöőüűÁÉÍÓÖŐÜŰ\s]', '', regex=True)

In [86]:
corpus = corpus[corpus.id != '']

In [87]:
corpus

,doc_id,id,text
29,0,Dávid,"- Közösen fogjuk olvasni, igen. De az a lény..."
30,0,A,- Jó.
32,0,D,"- Ha a gondolatot befejezted, akkor újba már ..."
37,0,Berta,- Én designképzésben szerettem volna végre ré...
38,0,D,"- Azt elkérhetem, mert ezeket majd így szépen..."
...,...,...,...
3974,7,J,"- Van egyébként erre lehetőség, én itt dolgozom."
3975,7,LE,- Az emaileket el kell olvasni és ott van benne.
3978,7,LE,"- De van, amelyiknek az a neve, hogy munkaleh..."
3989,7,J,"- Mondjuk, hogy így. Nekem van egy ilyen, de ..."


# Text preprocessing - bag of words

In [74]:
os.listdir('.\\data\\')

['2023 03 16 - MA más_szerkesztett.docx',
 '2023 03 23 - MA MOME_szerkesztett.docx',
 '2023 03 27 - BA3_szerkesztett.docx',
 '2023 03 28 - MA MOME (teljes)_szerkesztett.docx',
 '2023 03 29 - BA1_szerkesztett.docx',
 '2023 04 24 - BA1 (nem indult el időben - javított)_szerkesztett.docx',
 '2023 04 25 - BA3_szerkesztett.docx',
 '2023 04 25 - MA más_szerkesztett.docx']

In [73]:
'asdasdasd.\\'[-1:] == '\\'

True

In [46]:
import pandas as pd

In [47]:
import huspacy

nlp = huspacy.load('hu_core_news_lg')

In [48]:
from tqdm.notebook import tqdm

In [49]:
tqdm.pandas()

In [59]:
#stopszószűrés

In [58]:
stopwords = nlp.Defaults.stop_words

In [50]:
corpus.text = corpus.text.str.replace('[^a-zA-ZáéíóöőúüűÁÉÍÓÖŐÚÜŰ\s]', '', regex=True).str.lower().str.strip()

In [69]:
[tok for tok in corpus.text[32]]

['h',
 'a',
 ' ',
 'a',
 ' ',
 'g',
 'o',
 'n',
 'd',
 'o',
 'l',
 'a',
 't',
 'o',
 't',
 ' ',
 'b',
 'e',
 'f',
 'e',
 'j',
 'e',
 'z',
 't',
 'e',
 'd',
 ' ',
 'a',
 'k',
 'k',
 'o',
 'r',
 ' ',
 'ú',
 'j',
 'b',
 'a',
 ' ',
 'm',
 'á',
 'r',
 ' ',
 'n',
 'e',
 ' ',
 'k',
 'e',
 'z',
 'd',
 'j',
 ' ',
 'b',
 'e',
 'l',
 'e']

In [70]:
corpus.text.progress_apply(lambda text: ' '.join([tok for tok in text.split() if tok not in stopwords]))

  0%|          | 0/2773 [00:00<?, ?it/s]

29      közösen fogjuk olvasni lényeg tudd olvasni els...
30                                                       
32                  gondolatot befejezted újba kezdj bele
37      designképzésben szerettem végre részesülni ba bsc
38                      elkérhetem ezeket szépen kirakjuk
                              ...                        
3974                         egyébként lehetőség dolgozom
3975                                    emaileket olvasni
3978    amelyiknek neve munkalehetősége erről pont sze...
3989    mondjuk mondjuk szerintem feltétlenül mome hib...
3990    köszi szépen csoporton visszatérő probléma úgy...
Name: text, Length: 2773, dtype: object

In [52]:
corpus['docs']
corpus.text.progress_apply(lambda x: nlp(x))

  0%|          | 0/2773 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [75]:
corpus

,doc_id,id,text
29,0,Dávid,közösen fogjuk olvasni igen de az a lényeg ho...
30,0,A,jó
32,0,D,ha a gondolatot befejezted akkor újba már ne k...
37,0,Berta,én designképzésben szerettem volna végre része...
38,0,D,azt elkérhetem mert ezeket majd így szépen kir...
...,...,...,...
3974,7,J,van egyébként erre lehetőség én itt dolgozom
3975,7,LE,az emaileket el kell olvasni és ott van benne
3978,7,LE,de van amelyiknek az a neve hogy munkalehetősé...
3989,7,J,mondjuk hogy így nekem van egy ilyen de ez mon...
